Build the Neural Network
========================
[torch.nn](https://pytorch.org/docs/stable/nn.html)
命名空间提供了构建神经网络所需的所有基本模块
网络。PyTorch中的每个模块都是
[nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)
的子类。神经网络本身是由子模块(层)组成的模块，这种嵌套结构使我们能较为轻松地构建和管理复杂的网络结构。

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

D:\Anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


指定训练设备
=======================

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


定义神经网络类
================

我们通过子类化`nn.Module`来定义我们的神经网络，在
`__init__`函数中定义神经网络层。
每个`nn.Module`子类在`forward`方法中实现对输入数据的操作。

-   [torch.nn API](https://pytorch.org/docs/stable/nn.html)

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)
Predicted class: tensor([7], device='cuda:0')


模型参数
================
子类化`nn.Module`会自动跟踪模型对象内部定义的所有字段，并可以使用
`parameters()`或`named_parameters()`方法访问模型的参数。

In [4]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")




Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0039,  0.0240, -0.0252,  ..., -0.0232, -0.0298, -0.0304],
        [ 0.0043,  0.0258, -0.0353,  ...,  0.0026,  0.0060,  0.0008]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0197, -0.0038], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0417,  0.0218, -0.0409,  ..., -0.0227,  0.0407,  0.0337],
        [ 0.0121, -0.0376,  0.0028,  ..., -0.0416,  0.0144,  0.0057]],
       device='cuda:0', grad_fn=<Slic